In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import glob
import pickle
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from models import CNN_LSTM, SepCNN_LSTM, ConvGRU_LSTM, RandomForestBaseline, LassoModel
import joblib
from torchsummary import summary

In [2]:
seed = 42

# Set seed for NumPy
np.random.seed(seed)

# Set seed for Python's built-in random module
random.seed(seed)

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for Torch's CUDA operations if GPU is used
# if torch.cuda.is_available():
#     torch.backends.cudnn.deterministic = True
#     torch.cuda.manual_seed(seed)

In [3]:
# # Load a sample of the data
sample_data = np.load('./data/PROCESSED_III/2018_13_155.npy')  

# # Check the shape of the sample data
print("Shape of sample data:", sample_data.shape)

Shape of sample data: (38, 1, 128, 9)


In [4]:
# Define generator function
def generator(IDs, yields, batch_size, cutoff=None):
    def load_data(ID):
        try:
            data = np.load('./data/PROCESSED_III/' + ID + '.npy')
            return data, True
        except Exception as e:
            return None, False

    batches = 0

    while True:
        batch_features = np.zeros((batch_size, 38, 1, 128, 9)) if cutoff is None else np.zeros((batch_size, cutoff, 1, 128, 9))
        batch_yields = np.zeros(batch_size)

        if batches == len(IDs) // batch_size:
            batches = 0
            yield None, None

        for i in range(batch_size):
            while True:
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                data, success = load_data(ID)
                if success:
                    break

            if data is not None:
                if cutoff is not None:
                    if not np.isnan(data).any():
                        batch_features[i, :, :, :, :] = data[:cutoff, :, :, :]
                        batch_yields[i] = yields[ID]
                else:
                    batch_features[i, :, :, :, :] = data
                    batch_yields[i] = yields[ID]
                

        batches += 1

        yield torch.tensor(batch_features, dtype=torch.float32, device='cuda'), torch.tensor(batch_yields, dtype=torch.float32, device='cuda')


In [5]:
# Datasets
yields = pickle.load(open('data/yields.p', 'rb'))

# Generators
training_generator = generator(list(yields['train'].keys()), yields['train'], 16)
validation_generator = generator(list(yields['validation'].keys()), yields['validation'], 16)

In [5]:
n_estimators_values = [50, 100, 150]  
max_depth_values = [None, 10, 20]

# Initialize variables to store best parameters and corresponding MSE
best_params = None
best_mse = float('inf')  

for n_estimators in n_estimators_values:
    for max_depth in max_depth_values:
        random_forest_model = RandomForestBaseline(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

        # Fit the model to training data
        X_train, y_train = next(training_generator)
        random_forest_model.fit(X_train.cpu().reshape(X_train.shape[0], -1), y_train.cpu())

        # Make predictions on test data
        X_test, y_test = next(validation_generator)
        predictions = random_forest_model.predict(X_test.cpu().reshape(X_test.shape[0], -1))

        # Evaluate the model
        mse = random_forest_model.evaluate(X_test.cpu().reshape(X_test.cpu().shape[0], -1), y_test.cpu())

        # Print MSE for current parameter combination
        print(f"Parameters: n_estimators={n_estimators}, max_depth={max_depth} MSE: {mse}")

        # Check if current combination improves performance
        if mse < best_mse:
            print('save best model')
            joblib.dump(random_forest_model, 'random_forest_best_model.pkl')            
            best_mse = mse
            best_params = {'n_estimators': n_estimators, 'max_depth': max_depth}

# Print best parameters and corresponding MSE
print("\nBest Parameters:")
print(best_params)
print("Best Mean Squared Error:", best_mse)

Parameters: n_estimators=50, max_depth=None MSE: 1215.3222280938926
save best model
Parameters: n_estimators=50, max_depth=10 MSE: 1632.7786611852525
Parameters: n_estimators=50, max_depth=20 MSE: 1590.7839759603144
Parameters: n_estimators=100, max_depth=None MSE: 1881.0676816275231
Parameters: n_estimators=100, max_depth=10 MSE: 1404.0375377720136
Parameters: n_estimators=100, max_depth=20 MSE: 853.107621468707
save best model
Parameters: n_estimators=150, max_depth=None MSE: 1889.1073206777735
Parameters: n_estimators=150, max_depth=10 MSE: 1652.2194739090282
Parameters: n_estimators=150, max_depth=20 MSE: 2248.1081579971983

Best Parameters:
{'n_estimators': 100, 'max_depth': 20}
Best Mean Squared Error: 853.107621468707


In [6]:
alpha_values = [0.3, 0.4, 0.5]

best_alpha = None
best_mse = float('inf') 

# Iterate over alpha values
for alpha in alpha_values:
    # Initialize Lasso model with current alpha value
    lasso_model = LassoModel(alpha=alpha, random_state=42)

    # Fit the model to training data
    X_train, y_train = next(training_generator)
    lasso_model.fit(X_train.cpu().reshape(X_train.shape[0], -1), y_train.cpu())

    # Make predictions on test data
    X_test, y_test = next(validation_generator)
    predictions = lasso_model.predict(X_test.cpu().reshape(X_test.shape[0], -1))

    # Evaluate the model
    mse = lasso_model.evaluate(X_test.cpu().reshape(X_test.cpu().shape[0], -1), y_test.cpu())

    print(f"Alpha: {alpha}, MSE: {mse}")

    # Check if current alpha improves performance
    if mse < best_mse:
        print('save best model')
        joblib.dump(lasso_model, 'lasso_best_model.pkl')   
        best_mse = mse
        best_alpha = alpha

# Print best alpha and corresponding MSE
print("\nBest Alpha:", best_alpha)
print("Best Mean Squared Error:", best_mse)

Alpha: 0.3, MSE: 279.69549731154285
save best model
Alpha: 0.4, MSE: 2428.617348022587
Alpha: 0.5, MSE: 634.4768458833705

Best Alpha: 0.3
Best Mean Squared Error: 279.69549731154285


In [6]:
model_functions = {
    'CNN_LSTM': CNN_LSTM,
    'SepCNN_LSTM': SepCNN_LSTM,
    'ConvGRU_LSTM': ConvGRU_LSTM,
}


epochs = 100

for model_name, model_function in model_functions.items():
    model = model_function(dimensions=[38, 1, 128, 9])
    model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    best_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch_data, batch_labels in tqdm.tqdm(training_generator, desc=f"Epoch {epoch+1}/{epochs}"):
            if batch_data is None:
                break

            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for val_data, val_labels in validation_generator:
                if val_data is None:
                    break
                val_outputs = model(val_data)
                val_loss = criterion(val_outputs, val_labels.unsqueeze(1))
                val_losses.append(val_loss.item())

        current_loss = np.mean(val_losses)
        if current_loss < best_loss:
            torch.save(model, f'{model_name}_best.pt')
            best_loss = current_loss
            
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {current_loss:.4f}")

Epoch 1/100: 0it [00:00, ?it/s]d:\Miniconda\envs\sds\Lib\site-packages\torch\nn\modules\conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ..\aten\src\ATen\native\Convolution.cpp:1004.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch 1/100: 389it [00:27, 13.99it/s]


Epoch 1/100, Train Loss: 22225.5468, Val Loss: 20560.4383


Epoch 2/100: 389it [00:26, 14.88it/s]


Epoch 2/100, Train Loss: 12500.0966, Val Loss: 10456.9597


Epoch 3/100: 389it [00:26, 14.61it/s]


...

Epoch 98/100, Train Loss: 528.3994, Val Loss: 1050.8370


Epoch 99/100: 389it [01:13,  5.32it/s]


Epoch 99/100, Train Loss: 550.5661, Val Loss: 1107.5980


Epoch 100/100: 389it [01:12,  5.33it/s]


Epoch 100/100, Train Loss: 531.9821, Val Loss: 1009.0650


In [ ]:
trained_model = torch.load('./models/CNN_LSTM_best.pt')

test_gen = generator(list(yields['validation'].keys()), yields['validation'], len(yields['validation']))
X_test, y_test = next(test_gen)

In [9]:
lst = list(yields['train'].keys())
data_years = {
    '2018': [],
    '2019': [],
    '2020': [],
    '2021': [],
    '2022': [],
    '2023': [],
}

for data in lst:
    year = data.split('_')[0]
    data_years[year].append(data)

lst = list(yields['validation'].keys())

for data in lst:
    year = data.split('_')[0]
    data_years[year].append(data)

In [16]:
models = ['CNN_LSTM_Adam_lr_0.001', 'CNN_LSTM_Adam_lr_0.005', 'ConvGRU_LSTM_Adam_lr_0.001', 'ConvGRU_LSTM_Adam_lr_0.005']

all_data = {}

criterion = nn.MSELoss()

for model_name in models:
    if 'CNN_LSTM' in model_name:
        model = CNN_LSTM(dimensions=[38, 1, 128, 9])
    else:
        model = ConvGRU_LSTM(dimensions=[38, 1, 128, 9])
    model.load_state_dict(torch.load(f'./models/{model_name}_best.pt'))
    model.to('cuda')
    model.eval()

    for year in ['2018', '2019', '2020', '2021']:
        losses = []
        outputs = []

        if year == '2021':
            year_gen = generator(data_years[year], yields['validation'], 16)
        else:
            year_gen = generator(data_years[year], yields['train'], 16)

        with torch.no_grad():
            for data, yield_data in year_gen:
                if data is None:
                    break
                output = model(data)
                loss = criterion(output, yield_data.unsqueeze(1))
                outputs.append(output.cpu().numpy())
                losses.append(loss.item())

        all_data[f'{model_name}_{year}'] = (np.mean(losses), np.sum(outputs))

all_data

{'CNN_LSTM_Adam_lr_0.001_2018': (295.3537059474636, 184468.12),
 'CNN_LSTM_Adam_lr_0.001_2019': (243.38821160303402, 173044.56),
 'CNN_LSTM_Adam_lr_0.001_2020': (715.7331414694314, 220489.1),
 'CNN_LSTM_Adam_lr_0.001_2021': (1304.5928936004639, 187200.78),
 'CNN_LSTM_Adam_lr_0.005_2018': (557.2617604023701, 188240.3),
 'CNN_LSTM_Adam_lr_0.005_2019': (734.1392391675139, 178598.97),
 'CNN_LSTM_Adam_lr_0.005_2020': (452.00859891451324, 228300.23),
 'CNN_LSTM_Adam_lr_0.005_2021': (1417.686783027649, 198523.88),
 'ConvGRU_LSTM_Adam_lr_0.001_2018': (397.7488078555545, 191843.06),
 'ConvGRU_LSTM_Adam_lr_0.001_2019': (397.0032612003692, 182157.72),
 'ConvGRU_LSTM_Adam_lr_0.001_2020': (351.8298935104202, 230499.19),
 'ConvGRU_LSTM_Adam_lr_0.001_2021': (1193.4569580078125, 195139.4),
 'ConvGRU_LSTM_Adam_lr_0.005_2018': (530.0325443164722, 187800.28),
 'ConvGRU_LSTM_Adam_lr_0.005_2019': (577.3304587586285, 175580.12),
 'ConvGRU_LSTM_Adam_lr_0.005_2020': (410.55387308309366, 225286.42),
 'ConvGRU_